#  Instrument variable

### Økonometri A (ØkA)

Bertel Schjerning

Department of Economics, University of Copenhagen


### Enable autoreload

In [1]:
# Sørger for at alle importerede python filer geninlæses ved import statements
# Nødvedigt, hvis ændinger skal tage effekt uden at genstarte Python Kernel
%load_ext autoreload
%autoreload 2

# Part 1: IV estimation (løn regression)

### Indlæs data og moduler

In [20]:
import pandas as pd
import numpy as np
import mymlr as mlr  

# Indlæs data og opret variabler
df_mroz = pd.read_stata("../data/mroz.dta")   # Indlæs data fra en Stata-fil
df_mroz['const'] = 1  # Tilføj en konstant term (intercept)
df_mroz['lwage'] = np.log(df_mroz['wage'])  # Logaritmer lønvariablen 'wage'
df = df_mroz.dropna(subset='lwage')  # Fjern rækker med manglende værdier i 'lwage'

### Sammenlign OLS og IV

In [22]:
# OLS
m_ols = mlr.ols(df[['const', 'educ']], df['lwage'])  # Estimér OLS-model

# IVE med 'motheduc' som instrument
m_2sls = mlr.tsls(y=df['lwage'], 
                  X1=df[['const']], 
                  X2=df[['educ']], 
                  Ze=df[['motheduc']])  

mlr.summary([m_ols, m_2sls], column_titles=["OLS", "IV"], report_stats="t")


                         OLS       IV
Dependent variable     lwage    lwage
             const   -0.1852   0.7022
                   (-0.9998) (1.3984)
              educ    0.1086   0.0385
                    (7.5451) (0.9742)
         R_squared    0.1179   0.0022
               TSS  223.3274 223.3274
               RSS  197.0010 222.8310
               ESS   26.3264   0.4964
                 n       428      428
Note: t-statistics are reported in parentheses.



### OLS/IV: Simpel implemtering med en forklarende variable og et instrument 

In [77]:
cov=df[['lwage','educ', 'motheduc']].cov()
b_OLS=cov.loc['lwage', 'motheduc']/cov.loc['educ','educ']
b_IV=cov.loc['lwage', 'motheduc']/cov.loc['educ', 'motheduc']
print(f'Covariance matrix\n {cov}\n')
print(f'b_OLS (educ) {b_OLS.round(4)}')
print(f'b_IV  (educ) {b_IV.round(4)}')
mlr.summary([m_ols, m_2sls], column_titles=["OLS", "IV"], report_stats="t")


Covariance matrix
              lwage      educ   motheduc
lwage     0.523015  0.567466   0.112796
educ      0.567466  5.222942   2.925967
motheduc  0.112796  2.925967  10.943526

b_OLS (educ) 0.0216
b_IV  (educ) 0.0385
                         OLS       IV
Dependent variable     lwage    lwage
             const   -0.1852   0.7022
                   (-0.9998) (1.3984)
              educ    0.1086   0.0385
                    (7.5451) (0.9742)
         R_squared    0.1179   0.0022
               TSS  223.3274 223.3274
               RSS  197.0010 222.8310
               ESS   26.3264   0.4964
                 n       428      428
Note: t-statistics are reported in parentheses.



### IV som effekten af $z$ på $y$ relativt til effekten af $z$ på $x$

In [87]:
# First stage (effekt af z på x)
m_first = mlr.ols(y=df['educ'], X=df[['const', 'motheduc']])  

# Reduceret form (effekt af z på y)
m_reduced = mlr.ols(y=df['lwage'], X=df[['const', 'motheduc']])  

b_2SLS=m_reduced['beta_hat'][1]/m_first['beta_hat'][1]
print(f'b_IV   (educ) {b_IV.round(4)}')
print(f'b_2SLS (educ) {b_2SLS[0].round(4)}')


b_IV   (educ) 0.0385
b_2SLS (educ) 0.0385


In [ ]:
* Reduceret form
reg logwage motheduc if logwage ~= .
global b_reduced = _b[motheduc]

* First stage
reg educ motheduc if logwage ~= .
global b_first = _b[motheduc]

global b_IV = $b_reduced / $b_first
display $b_IV

* IV estimation
ivregress 2sls logwage (educ=motheduc) if logwage ~= .

# Part 2: Estimation af model for consumer demand

Grieco, P.L., Murry, C. and Yurukoglu, A. (2024): "The evolution of market power in the us automobile industry."
<br>*The Quarterly Journal of Economics*, 139(2), pp.1201-1253.


## Modellering af Markedsandele: Logit Model

**Nyttefunktion for Forbruger $i$, Produkt $j$ i Marked $m$:**

$$
U_{imj} = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj} + \epsilon_{imj}
$$

- $\mathbf{x}_{mj}$: Observerbare karakteristika for produkt $j$ i marked $m$ (f.eks. størrelse, kvalitet).
- $p_{mj}$: Pris for produkt $j$ i marked $m$.
- $\xi_{mj}$: Uobserverbare karakteristika (f.eks. brand, placering).
- $\epsilon_{imj}$: Idiosynkratisk fejlled, der følger en ekstremværdi fordeling (Gumbel).

**Markedsandel for Produkt $j$ i Marked $m$:**

Forbrugeren vælger produktet, der maksimerer nytte, hvilket giver beslutningsreglen:
$$
j_{im}^* = \arg \max_{j=0,1,\dots, J} U_{imj}
$$

**Markedsandele:**

I Økonometri B viser vi, at antagelsen om, at $\epsilon_{imj}$ er ekstremværdi fordelt,
leder til markedsandele på "logit form":
$$
S_{mj} = \frac{\exp(U_{imj})}{\sum_{k=0}^{J} \exp(U_{imk})}
$$

$S_{mj}$ er sandsynligheden for, at forbruger $i$ vælger produkt $j$ i marked $m$.

## Modellering af Markedsandele: Log-Linearisering

**Markedsandel for Produkt $j$ i Marked $m$:**
$$
S_{mj} = \frac{\exp(U_{imj})}{\sum_{k=0}^{J} \exp(U_{imk})}
$$

**Log-markedsandel for Produkt $j$:**
$$
\log(S_{mj}) = U_{imj} - \log\left( \sum_{k=0}^{J} \exp(U_{imk}) \right) = U_{imj} - \Lambda_{im}
$$

**Log-differens af markedsandele mellem to produkter $j$ og $k$:**
$$
\log\left(\frac{S_{mj}}{S_{mk}}\right) =  U_{imj} - \Lambda_{im} - \left( U_{imk} - \Lambda_{im} \right) = U_{imj} - U_{imk}
$$

- Log-summen forsvinder når vi tager forskellen mellem produkterne.
- Log-summen $\Lambda_{im} = \log\left( \sum_{k=0}^{J} \exp(U_{imk}) \right)$ er uafhængig af $j$ og $k$, da vi summerer over alle produkter i markedet.

**Dette resulterer i en lineær model:**
$$
\log\left(\frac{S_{mj}}{S_{mk}}\right) = \beta(\mathbf{x}_{mj} - \mathbf{x}_{mk}) - \alpha(p_{mj} - p_{mk}) + (\xi_{mj} - \xi_{mk})
$$

## Normalisering af Nytte for Outside Good

**Outside Good (ingen bilkøb):**
- For det outside-alternativ, hvor forbrugeren vælger ikke at købe en bil, antager vi:
$$
U_{im0} = 0
$$
- Da vi ikke observerer prisen eller karakteristika for outside good, gør denne normalisering det muligt at sammenligne nytten af andre produkter relativt til ikke-køb.

**Normaliseret nytte for produkt $j$:**
- Nyttefunktionen for produkt $j$ bliver:
$$
U_{imj} = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj}
$$

**Markedsandel relativt til outside good:**
- Ved at tage log-forskellen mellem markedsandelen for produkt $j$ og outside good ($S_{m0}$), får vi:
$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj}
$$

- Normaliseringen sætter outside good's nytte til 0, hvilket resulterer i en lineær model, hvor $\alpha$ måler den marginale nytte af penge, og $\beta$ måler den marginale effekt af produktets karakteristika.

### Indlæs relevante libraries 

In [ ]:
import pandas as pd
import numpy as np
import mymlr as mlr # see mymlr.py
pd.set_option('display.float_format', '{:.8f}'.format)

### Indlæs data

In [ ]:
# Indlæs datasæt
df = pd.read_csv("../data/cars.csv")
df['const'] = 1 # tilføj konstant
df["market_ids"] = df["year"] # Definér "market_ids" som året for salget
pd.set_option('display.max_columns', None)  # Vis alle kolonner
pd.set_option('display.max_rows',20) # Begræns til 20 rækker
display(df.round(4)) # Vis datasættet med nye kolonner

### Forbered variable til estimation

$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj}
$$

Først venstresiden 
$$
\text{dlogS}=\log\left(\frac{S_{mj}}{S_{m0}}\right)
$$

In [ ]:
# Beregn markedsandele Smj baseret på antagelsen om, 
# - at hver husholdning har 2 biler
# - og køber en ny bil hvert 2.5 år
df["Smj"] = df.apply(lambda x: x["sales"] / (x["number_households"] / 2.5), axis=1)

# Beregn andelen af husholdninger uden ny bil, baseret på de beregnede Smj værdier
Sm0 = 1 - df.groupby("year")["Smj"].sum() 

# Forskellen i log-markedsandel mellem en given bil og andelen uden ny bil
df["dlogS"] = df.apply(lambda x: np.log(x["Smj"]) - np.log(Sm0[x["year"]]), axis=1)

### Forbered variable til estimation

$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj}
$$

Nu de forklarende variable 
- $\mathbf{x}_{mj}$: Observerbare karakteristika for produkt $j$ i marked $m$: 
    
        "log_height","log_footprint","log_hp","log_mpg",
        "log_curbweight","log_number_trims",
         "releaseYear","yearsSinceDesign"
- $p_{mj}$: Pris for produkt $j$ i marked $m$.
    - manufacturer suggested retail price skaleret fra $1000$ til $10.000$
            "msrp" x 0.1

In [ ]:
# Priser, pmj
df["prices"] = df["msrp"].copy()*0.1 # Skaler priserne fra 1000$ til 10.000$

# Liste med x variable (vehicle attributes, xmj) - variable findes i df
Xvar = ["log_height","log_footprint","log_hp","log_mpg",
        "log_curbweight","log_number_trims",
       "releaseYear","yearsSinceDesign"] 

### Forbered variable til estimation
Tilføj dummies som proxy for $\xi_{mj}$
$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj} 
$$
Dummies for
- **biltyper** (sport, EV, truck, suv, van) 
- **kalender år** (reference kategori: $1980$)
- **bilmærke** (reference kategori: Volvo)

### Dummies

In [ ]:
# Liste med dummy variable for biltyper - findes allerede i df
D_type= ["sport","EV","truck","suv","van"] 

# Opret dummies for år og fjern 1980 som referencekategori, konverter til int for sikkerhed
year_dummies = pd.get_dummies(df["year"], prefix='y').astype(int)

# Opret dummies for bilproducenter og fjern Volvo som referencekategori
make_dummies = pd.get_dummies(df["make"]).astype(int)

# Kolonnenavne for dummies
D_year = year_dummies.columns.tolist()  # Konverter Int64Index til en liste af strenge
D_make = make_dummies.columns.tolist()  # Konverter Int64Index til en liste af strenge
D_year.remove("y_1981") # reference kategori er 1980
D_make.remove("volvo") # reference kategori er 1980

# Tilføj gem df og dummeies i ny dataframe, X
X = pd.concat([df, year_dummies, make_dummies], axis=1)
X[D_make].head(100)
X[D_make].sum(axis=0)

In [ ]:
import matplotlib.pyplot as plt

# Beregn summen af dummyvariablerne og tilføj reference kategorien (sum må ikke være 0)
sums = X[D_type].sum(axis=0) # e.g. type, make, year
sums["reference kategori"]=X["const"].sum()-sums.sum()

# Lav en søjlediagram for summen 
sums.plot(kind='bar', figsize=(10, 6))

# Tilføj labels og titel
plt.ylabel('Antal obs')
plt.xticks(rotation=90)  # Drej x-axis labels for bedre læsbarhed

# Vis diagrammet
plt.show()

## Modelspecifikation for forbrugerens efterspørgsel

$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta \mathbf{x}_{jm} - \alpha p_{jm} + \xi_{jm}
$$

hvor:
- $m$ refererer til **markedet** (her **året**).
- $S_{jm}$: Markedsandel af produkt $j$ i marked $m$.
- $S_{0m}$: Markedsandel for det "outside good" (ingen bilkøb) i marked $m$.
- $\mathbf{x}_{jm}$: Observerbare produktkarakteristika.
- $p_{jm}$: Pris for produkt $j$ i marked $m$.
- $\xi_{jm}$: Uobserverbare produktkarakteristika.

Vi estimerer fem versioner af følgende model, som adskiller sig ved hvordan $\xi_{jm}$ approximeres

#### Model 1: Baseline specifikation med kun priser ($p_{jm}$):
$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta_0 + \beta_{\text{price}} p_{jm} + \epsilon_{jm}
$$

#### Model 2: Priser og produktkarakteristika ($\mathbf{x}_{jm}$):
$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta_0 + \beta_{\text{price}} p_{jm} + \sum \beta_k \mathbf{x}_{jm,k} + \epsilon_{jm}
$$

#### Model 3: Tilføjer dummies for biltyper (sport, EV, truck, SUV, van):
$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta_0 + \beta_{\text{price}} p_{jm} + \sum \beta_k \mathbf{x}_{jm,k} + \sum \delta_t D_{\text{type}, m} + \epsilon_{jm}
$$

#### Model 4: Tilføjer årsdummies for at kontrollere for markedsvirkninger:
$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta_0 + \beta_{\text{price}} p_{jm} + \sum \beta_k \mathbf{x}_{jm,k} + \sum \delta_t D_{\text{type}, m} + \sum \gamma_y D_{\text{year}, m} + \epsilon_{jm}
$$

#### Model 5: Fuldspecifikation med bilmærke-dummies (fixed effects for mærker):
$$
\log\left(\frac{S_{jm}}{S_{0m}}\right) = \beta_0 + \beta_{\text{price}} p_{jm} + \sum \beta_k \mathbf{x}_{jm,k} + \sum \delta_t D_{\text{type}, m} + \sum \gamma_y D_{\text{year}, m} + \sum \eta_m D_{\text{make}, m} + \epsilon_{jm}
$$

**Bemærk**: I denne opsætning svarer marked $m$ til året ($m = t$).

### Estimate model

In [ ]:
        
m1 = mlr.ols(X[['const','prices']], X["dlogS"] )
m2 = mlr.ols(X[['const','prices'] + Xvar], X["dlogS"] )
m3 = mlr.ols(X[['const','prices'] + Xvar + D_type], X["dlogS"] )
m4 = mlr.ols(X[['const','prices'] + Xvar + D_type + D_year], X["dlogS"] )
m5 = mlr.ols(X[['const','prices'] + Xvar + D_type + D_year + D_make], X["dlogS"] )

mlr.summary([m1,m2,m3, m4, m5])


## Fixed Effects og Within Transformation - alternativ til dummies

Vi antager, at en del af $\xi_{mj}$, det uobserverbare led i vores model, er specifikt for bilproducenten (mærket). Dette giver anledning til følgende lineære model:

$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \xi_{mj}
$$

### Opsplitning af $\xi_{mj}$:
Lad $\xi_{mj} = \gamma_{\text{make}} + \tilde{\xi}_{mj}$, hvor $\gamma_{\text{make}}$ repræsenterer fixed effect for producenten (mærket), og $\tilde{\xi}_{mj}$ er den resterende fejlled.

Dermed bliver modellen:
$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) = \beta \mathbf{x}_{mj} - \alpha p_{mj} + \gamma_{\text{make}} + \tilde{\xi}_{mj}
$$

### Anvendelse af Within Transformation:
For at eliminere fixed effect $\gamma_{\text{make}}$, de-meaner vi ligningen ved at trække gennemsnittet inden for hver producentgruppe fra:

$$
\log\left(\frac{S_{mj}}{S_{m0}}\right) - \overline{\log\left(\frac{S_{mj}}{S_{m0}}\right)} = \beta \left(\mathbf{x}_{mj} - \overline{\mathbf{x}_{mj}}\right) - \alpha \left(p_{mj} - \overline{p_{mj}}\right) + \left(\tilde{\xi}_{mj} - \overline{\tilde{\xi}_{mj}}\right)
$$

Da $\gamma_{\text{make}}$ er konstant inden for hver gruppe, forsvinder det ved transformationen:
$$
\log\left(\frac{S_{mj}}{S_{m0}}\right)^{\text{demeaned}} = \beta \mathbf{x}_{mj}^{\text{demeaned}} - \alpha p_{mj}^{\text{demeaned}} + \tilde{\xi}_{mj}^{\text{demeaned}}
$$

### Bemærk:
- **Fixed effects elimineres**, så vi kan estimere $\alpha$ og $\beta$ ved at bruge variationen inden for hver producent.
- Vi får **samme parametre** som ved inkludering af producent-dummies i modellen.
- **Demeaning** er en praktisk metode til at undgå at inkludere mange dummies i modellen.
- Dette er en anvendelse af **Frisch-Waugh-teoremet**, som viser, at vi kan regressere de-meanet $y$ på de-meanet $x$ og få de samme resultater som med dummies.

In [ ]:
# Function to perform within transformation (demean within groups)
def within_transform(df, columns, category):
    # Create a copy of the dataframe to avoid modifying the original
    df_transformed = df.copy()
    
    # Loop through each column that requires transformation
    for col in columns:
        # Compute group means based on the categorized variable
        group_means = df.groupby(category)[col].transform('mean')
        
        # Subtract group means from the column values (demeaning within groups)
        df_transformed[col] = df[col] - group_means
    
    return df_transformed

Zvars= ['prices'] + Xvar + D_type + D_year; 
Xt = within_transform(X, Zvars + ['dlogS'], 'make')

m_ols = mlr.ols(X[['const'] + Zvars + D_make], X["dlogS"] )
m_fe = mlr.ols(Xt[Zvars], Xt["dlogS"] )

mlr.summary([m_ols, m_fe])


## Endogenitetsproblemet: 
#### Er MLR.4 opfyldt? $$E[\xi_{jm}|\mathbf{x}_{jm},p_{jm}]=0$$

**Hvad er endogenitet?**
- Endogenitet opstår, når $[\xi_{jm}|\mathbf{x}_{jm},p_{jm}] \neq 0$
- I vores tilfælde kan prisen ($p_{mj}$) være endogen, hvis uobserverbare faktorer, der påvirker efterspørgslen ($\xi_{mj}$), også påvirker prisen.

**Eksempel:**
- **Uobserverede karakteristika ($\xi_{mj}$):** Ting som kvalitet, mærke-image, eller økonomiske stød kan påvirke både efterspørgslen og prisen.
- Dette betyder, at når vi forsøger at estimere effekten af prisen på markedsandele, vil vi fejlagtigt tilskrive en del af effekten fra $\xi_{mj}$ til prisen.

**Konsekvens:**
- Hvis der er endogenitet, vil OLS estimaterne af $\alpha$ (pris-effekten) være biased, da de uobserverbare faktorer ikke er uafhængige af prisen.

### Beregn egenpriselasticitet

For at beregne egenpriselasticiteten $\varepsilon_j$ bruger vi følgende udtryk baseret på estimaterne fra vores logit-model:

$$
\varepsilon_{mj} \equiv \frac{dS_{mj}/S_{mj}}{dp_{mj}/p_{mj}} = \alpha \cdot p_{mj} \cdot (1 - S_{mj})
$$

- Model 1: Baseline specifikation med kun priser ($p_{jm}$):
- Model 2: Priser og produktkarakteristika ($\mathbf{x}_{jm}$):
- Model 3: Tilføjer dummies for biltyper (sport, EV, truck, SUV, van):
- Model 4: Tilføjer årsdummies for at kontrollere for markedsvirkninger:
- Model 5: Fuldspecifikation med bilmærke-dummies (fixed effects for mærker):

In [ ]:
for i, model in enumerate([m1,m2,m3,m4,m5]):
    idx=model['lbl_X'].index('prices');
    alpha = model["beta_hat"][idx]
    X["elasticity"] = df.apply(lambda x: alpha * x["prices"] * (1 - x["Smj"]) , axis=1)
    print(f"Gennemsnitlig egenpris elasticitet for model {i+1} = {X['elasticity'].mean()}")

## Hvordan kan dummies reducere endogenitetsproblemet?

**Inkludering af dummies kan hjælpe ved at:**
- Kontrollere for **faste uobserverbare faktorer**, som er konstante over tid eller mellem mærker:
    - **Års-dummies:** Fanger makroøkonomiske forhold som inflation eller recessioner, der påvirker både pris og efterspørgsel i et givet år.
    - **Bilmærke-dummies:** Tager højde for bilspecifikke faktorer som brand image eller kvalitet, som kan være konstante over tid men variere mellem bilmærker.

**Hvordan dummies afhjælper endogenitet:**
- Dummies fjerner noget af den uobserverede variation, som påvirker både priser og markedsandele.
- Derved får vi renere estimater af effekten af prisen, da en del af den fælles variation mellem pris og $\xi_{mj}$ fjernes.

## Hvorfor er IVs nødvendige for at fjerne endogenitet helt?

**Dummies har begrænsninger:**
- Selvom dummies hjælper med at kontrollere for faste eller tidsvarierende uobserverede faktorer, kan de ikke fange **variation inden for en kategori**.
    - Eksempel: Selv med bilmærke-dummies kan vi ikke kontrollere for variation i efterspørgsel inden for BMW-biler (som fx model-specifikke udsving).
- Der kan stadig være en korrelation mellem priser og uobserverbare faktorer inden for hver kategori.

**Løsningen: Instrumentvariable (IVs):**
- IVs bruges til at eliminere endogenitet, når vi har variabler, der er korrelerede med prisen, men ikke direkte med fejlleddet ($u$).
    - **Eksempel:** Produktionsomkostninger, eller inputpriser, der påvirker prisen, men ikke efterspørgslen direkte.
- Ved at bruge IVs kan vi isolere den eksogene variation i prisen, hvilket fører til unbiased estimater af pris-effekten.
- Vi ser på IV estimation i næste uge

## Kommentarer på resultaterne

**Resultaterne viser:**
- Uden dummies (model 1), er priselasticiteten meget høj (-1.34), hvilket tyder på, at der kan være betydelig endogenitet, hvor uobserverede faktorer påvirker både prisen og efterspørgslen.
- Når vi tilføjer flere dummies (model 2 til model 5), falder priselasticiteten gradvist, hvilket indikerer, at kontrollen for uobserverede faktorer gennem dummies afhjælper en del af endogeniteten.
    - I model 5 er elasticiteten -1.21, hvilket er mere plausibelt, men vi kan stadig ikke udelukke, at der er yderligere endogenitet.

**Begrænsning uden IVs:**
- Selvom dummies har reduceret noget af endogeniteten, kan vi ikke fjerne al endogenitet uden IVs. For at få fuldstændigt unbiased estimater af priselasticiteten, skal vi inkludere eksterne instrumenter.

## Diskussion af Endogenitetsproblemet

Priser er endogene, da de både kan afhænge af efterspørgsel og uobserverede faktorer, såsom produktets kvalitet, der også påvirker efterspørgslen. Hvis vi ikke kontrollerer for endogenitet, kan vores estimater for prisen ($\alpha$) blive biased.

### Hvordan kan dummies/fixed effects hjælpe?
- Ved at inkludere dummies for mærker og år, kan vi fange uobserverede faktorer, som kan variere mellem producenter og over tid.
- Det hjælper med at eliminere noget af den fælles variation i pris, men fjerner ikke nødvendigvis endogenitet fuldstændigt, især når den varierer inden for en given kategori (f.eks. mellem biler fra samme producent eller år).

### IV-løsning
For fuldstændigt at eliminere endogenitetsproblemet, skal vi bruge instrumentelle variable (IVs). 
I denne artikel bruger forfatterne **inputomkostninger og valutakurser** som instrumenter, da de påvirker priserne, men ikke direkte efterspørgslen. Dette fjerner den del af prisvariationen, der er forårsaget af endogenitet, og fører til mere pålidelige estimater for prisfølsomheden.

### Resultater og implikationer
- I vores modeller ser vi, at inklusionen af flere variable og dummies reducerer prisfølsomheden. Dette indikerer, at endogenitet påvirker vores grundlæggende estimater. 
- IV-metoden, som bruges i artiklen, giver et næste skridt for at fjerne endogenitetsbias og sikre mere præcise resultater.